In [13]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [21]:
# Functions to read in the corpus
#这个表示很奇妙，会根据长度而自己增长, 以前没有见过这种用法
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      #tag记录了喜好厌恶程度
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [22]:
# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
#一旦遇见新词，就启用UNK，其他旧词沿用w2i的index
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [23]:
train

,
   19,
   1300,
   1329,
   37,
   1,
   1494,
   283,
   4250,
   33],
  0),
 ([4364, 26, 3314, 71, 310, 2514, 508, 33], 0),
 ([21,
   4365,
   14,
   67,
   4366,
   23,
   1,
   172,
   4367,
   26,
   4368,
   549,
   2759,
   3034,
   226,
   233,
   132,
   1864,
   484,
   83,
   19,
   2569,
   2347,
   4369,
   2047,
   37,
   19,
   4370,
   1258,
   33],
  3),
 ([1284,
   5,
   1,
   4371,
   9,
   1523,
   37,
   4372,
   26,
   1291,
   4373,
   14,
   481,
   2910,
   26,
   573,
   646,
   1442,
   14,
   4374,
   118,
   1,
   266,
   4375,
   3274,
   33],
  0),
 ([4376,
   14,
   461,
   114,
   4377,
   177,
   3430,
   89,
   5,
   4378,
   1,
   309,
   1624,
   343,
   4379,
   4380,
   1378,
   33],
  0),
 ([847,
   14,
   4381,
   167,
   19,
   4382,
   3338,
   26,
   4383,
   254,
   19,
   4384,
   26,
   4385,
   4386,
   14,
   4387,
   10,
   334,
   313,
   132,
   33],
  1),
 ([1, 1378, 37, 4388, 3984, 3, 353, 71, 4248, 2877, 33], 0),
 ([132,
   9,
  

In [24]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [25]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [26]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  b_sm_exp = dy.parameter(b_sm)
  return score + b_sm_exp

In [27]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
iter 0: train loss/sent=1.5183, time=0.32s
iter 0: test acc=0.3498
iter 1: train loss/sent=1.3388, time=0.28s
iter 1: test acc=0.3810
iter 2: train loss/sent=1.2361, time=0.28s
iter 2: test acc=0.3810
iter 3: train loss/sent=1.1569, time=0.31s
iter 3: test acc=0.4072
iter 4: train loss/sent=1.0923, time=0.32s
iter 4: test acc=0.3941
iter 5: train loss/sent=1.0381, time=0.29s
iter 5: test acc=0.4063
iter 6: train loss/sent=0.9913, time=0.29s
iter 6: test acc=0.4072
iter 7: train loss/sent=0.9498, time=0.28s
iter 7: test acc=0.3905
iter 8: train loss/sent=0.9134, time=0.28s
iter 8: test acc=0.4109
iter 9: train loss/sent=0.8796, time=0.29s
iter 9: test acc=0.4081
iter 10: train loss/sent=0.8504, time=0.27s
iter 10: test acc=0.4041
iter 11: train loss/sent=0.8229, time=0.28s
iter 11: test acc=0.4131
ite